In [11]:
import duckdb
from polars import DataFrame
from pathlib import Path

In [12]:
Path().absolute().parent.parent

PosixPath('/workspaces/VR-data')

# Alustetaan yhteyttö tietokantaan

In [13]:
project_dir = Path().absolute().parent.parent
db_path = project_dir / "data" / "warehouse" / "vr.duckdb"
print(str(db_path), "  file exists:  ", db_path.is_file())

/workspaces/VR-data/data/warehouse/vr.duckdb   file exists:   True


In [14]:
# funktio kyselyn tekemistä varten ei vaadi yhteyden sulkua koska käytetään WITH
def sql(query: str) -> DataFrame:
    with duckdb.connect(str(db_path), read_only=True) as conn:
        df = conn.execute(query).pl()
    return df


In [15]:
sql("SHOW ALL TABLES")

database,schema,name,column_names,column_types,temporary
str,str,str,list[str],list[str],bool
"""vr""","""medallion""","""my_first_dbt_m…","[""id""]","[""INTEGER""]",false
"""vr""","""medallion""","""my_second_dbt_…","[""id""]","[""INTEGER""]",false
"""vr""","""medallion_bron…","""method_b_train…","[""route_sk"", ""trainNumber"", … ""timeTableRows""]","[""VARCHAR"", ""BIGINT"", … ""INTEGER[]""]",false


In [17]:
sql("SHOW").select("database", "schema", "name")

database,schema,name
str,str,str
"""vr""","""medallion""","""my_first_dbt_m…"
"""vr""","""medallion""","""my_second_dbt_…"
"""vr""","""medallion_bron…","""method_b_train…"


# Databasen selailu Silver-tasoa varten

In [20]:
sql("""
WITH exploded AS (
    SELECT 
        route_sk,
        UNNEST(timeTableRows) as timetable_row
    FROM medallion_bronze.method_b_traintest
),

defined AS (
        SELECT 
        route_sk,
        timetable_row::STRUCT(
            stationShortCode STRING,
            stationUICCode INT,
            countryCode STRING,
            type STRING,
            trainStopping BOOLEAN,
            commercialStop BOOLEAN,
            commercialTrack STRING, 
            cancelled BOOLEAN,
            scheduledTime STRING,
            liveEstimateTime STRING,
            estimateSource STRING,
            unknownDelay BOOLEAN, 
            actualTime STRING,
            differenceInMinutes INT,
            causes STRUCT(
                passengerTerm STRING, 
                categoryCode STRING, 
                categoryName STRING, 
                validFrom STRING,
                validTo STRING, 
                id INT, 
                detailedCategoryCode STRING, 
                detailedCategoryName STRING,
                thirdCategoryCode STRING, 
                thirdCategoryName STRING, 
                description STRING,
                categoryCodeId INT, 
                detailedCategoryCodeId INT, 
                thirdCategoryCodeId INT)[],
            trainReady STRUCT(
                source STRING,
                accepted BOOLEAN,
                timestamp STRING)[]
        ) as timetable_struct
    FROM exploded
),
    
flattened AS(
    SELECT 
        md5(route_sk || timetable_struct.stationShortCode || timetable_struct.stationUICCode) as stop_sk,
        route_sk,
        timetable_struct.*
    FROM defined
)

SELECT    
    stop_sk,    
    FIRST(stationShortCode),    
    FIRST(stationUICCode) as station_id,      
    FLATTEN(LIST(causes)) as lateness_causes
FROM flattened
GROUP BY stop_sk
""")

stop_sk,first(stationShortCode),station_id,lateness_causes
str,str,i32,list[struct[14]]
